# Llama Packs 示例

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llama_hub/llama_packs_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>

这个示例向您展示了如何在VoyageAI中使用一个简单的Llama Pack。我们将展示以下内容：
- 如何下载一个Llama Pack
- 如何检查其模块
- 如何直接运行它
- 如何自定义它。

您可以在 https://llamahub.ai 找到所有的packs。


```python
import pandas as pd

# 读取数据
data = pd.read_csv('data.csv')

# 显示数据的前5行
data.head()
```


In [ ]:
!wget "https://www.dropbox.com/s/f6bmb19xdg0xedm/paul_graham_essay.txt?dl=1" -O paul_graham_essay.txt

In [ ]:
from llama_index.core import SimpleDirectoryReader

# 加载一些示例数据
reader = SimpleDirectoryReader(input_files=["paul_graham_essay.txt"])
documents = reader.load_data()

### 下载和初始化包

我们使用 `download_llama_pack` 来下载包类，然后我们用文档初始化它。

每个包将具有不同的初始化参数。您可以通过其[README](https://github.com/logan-markewich/llama-hub/tree/main/llama_hub/llama_packs/voyage_query_engine)（也在LlamaHub上）找到有关每个包的初始化参数的更多信息。

**注意**：您还必须指定一个输出目录。在这种情况下，包被下载到 `voyage_pack`。这样可以让您自定义并对文件进行更改，并在以后导入它！


In [ ]:
from llama_index.core.llama_pack import download_llama_pack

VoyageQueryEnginePack = download_llama_pack(
    "VoyageQueryEnginePack", "./voyage_pack"
)

In [ ]:
voyage_pack = VoyageQueryEnginePack(documents)

### 检查模块


In [ ]:
modules = voyage_pack.get_modules()
display(modules)

In [ ]:
llm = modules["llm"]
vector_index = modules["index"]

In [ ]:
# 尝试使用LLM
response = llm.complete("你好，世界")
print(str(response))

In [ ]:
# 尝试检索器
retriever = vector_index.as_retriever()
results = retriever.retrieve("作者在成长过程中做了什么？")
print(str(results[0].get_content()))

### 运行包

每个包都有一个`run`函数，可以直接完成特定的任务。在这里，我们将使用VoyageAI嵌入来完整运行RAG流程。


In [ ]:
# 这将运行完整的包
response = voyage_pack.run(
    "作者在成长过程中做了什么？", similarity_top_k=2
)

In [ ]:
print(str(response))

The author spent his time outside of school mainly writing and programming. He wrote short stories and attempted to write programs on an IBM 1401. Later, he started programming on a TRS-80, creating simple games and a word processor. He also painted still lives while studying at the Accademia.


### 尝试自定义包

LlamaPacks的一个重要特性是您可以并且应该检查和修改代码模板！

在这个示例中，我们将展示如何使用不同的LLM自定义模板，同时保留Voyage嵌入，并且可以重复使用它。我们将使用Anthropic。

让我们进入`voyage_pack`并创建一个副本。

1. 为了演示目的，我们将`voyage_pack`复制到`voyage_pack_copy`。
2. 进入`voyage_pack_copy/base.py`并查看`VoyageQueryEnginePack`类的定义。这是所有核心逻辑所在的地方。正如您所看到的，包类本身是一个非常轻量级的基本抽象。您可以自由地按照您的意愿复制/粘贴代码。
3. 进入`__init__`中执行`llm = OpenAI(model="gpt-4")`的行，并将其改为`llm = Anthropic()`（默认为claude-2）。
4. 执行`from llama_index.llms import Anthropic`，并确保您的环境变量中设置了`ANTHROPIC_API_KEY`。
5. 现在您可以使用了！

在下面的部分中，我们将直接重新导入修改后的`VoyageQueryEnginePack`并使用它。


In [ ]:
from voyage_pack_copy.base import VoyageQueryEnginePack

voyage_pack = VoyageQueryEnginePack(documents)

In [ ]:
response = voyage_pack.run("What did the author do during his time in RISD?")
print(str(response))

 Unfortunately I do not have enough context in the provided information to definitively state what the author did during his time at RISD. The passage mentions that he learned a lot in a color class he took there, that he was basically teaching himself to paint, and that in 1993 he dropped out. But there are no specific details provided about his activities or course of study during his time enrolled at RISD. I apologize that I cannot provide a more complete response.
